In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.spatial.transform import Rotation as Rot
from mpl_toolkits.mplot3d import Axes3D
from liftpose.plot import plot_pose_3d, plot_pose_2d
import pickle
import matplotlib.pyplot as plt
import math
import matplotlib.animation as manimation
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load sample data

In [2]:
#load 3D poses
par_data = yaml.full_load(open('param.yaml', "rb"))
d = pickle.load(open('pose.pkl', 'rb'))
dimensions = [i for i in range(38) if i not in [15,16,17,18,34,35,36,37]]
pts3d = d["points3d"][171][dimensions,:]
ptssh = d["points2d"][0,171,dimensions,:]

pts3d = pts3d - np.mean(pts3d, axis=0, keepdims=True)

In [3]:
%matplotlib notebook
from ipywidgets import interact
import ipywidgets as widgets

def cone(alpha1, alpha2, beta):
    
    radius = 2
    theta = np.linspace(alpha1,alpha2,50)
    r = np.linspace(-radius,radius,50)
    T, R = np.meshgrid(theta, r)
    
    X = np.sign(R)* R * np.cos(T)
    Y = np.sign(R)* R * np.sin(T)
    
    Xplus = X.copy()
    Xminus = X.copy()
    Yplus = Y.copy()
    Yminus = Y.copy() 
    Xplus[R<=0] = 0
    Xminus[R>=0] = 0
    Yplus[R<=0] = 0
    Yminus[R>=0] = 0
    
    Z = np.tan(beta)*np.sqrt(Xplus**2 + Yplus**2)- np.tan(beta)*np.sqrt(Xminus**2 + Yminus**2)
    
    z = np.linspace(-radius*np.tan(beta), radius*np.tan(beta), 50)
    U, V = np.meshgrid(z, theta)

    Zc = U
    Xc = radius*np.cos(V)
    Yc = radius*np.sin(V)
    
    return X,Y,Z, Xc,Yc,Zc

def three_d_plotter(offset, alphaL1, alphaL2, alphaR1, alphaR2, beta):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(131, projection="3d")
    ax1.view_init(elev=50, azim=-30)
    ax2 = fig.add_subplot(132, projection="3d")
    ax2.view_init(elev=0, azim=0)
    ax3 = fig.add_subplot(133, projection="3d")
    ax3.view_init(elev=90, azim=90)
    
    #left cone
    X,Y,Z, Xc,Yc,Zc = cone(np.radians(alphaL1), np.radians(alphaL2), np.radians(beta))
    ax1.plot_surface(X,Y,Z, alpha=0.3, color='blue', rstride=6, cstride=12)
    ax2.plot_surface(X,Y,Z, alpha=0.3, color='blue', rstride=6, cstride=12)
    ax3.plot_surface(X,Y,Z, alpha=0.3, color='blue', rstride=6, cstride=12)
    ax1.plot_surface(Xc, Yc, Zc, alpha=0.3, color='blue', rstride=6, cstride=12)
    ax2.plot_surface(Xc, Yc, Zc, alpha=0.3, color='blue', rstride=6, cstride=12)
    ax3.plot_surface(Xc, Yc, Zc, alpha=0.3, color='blue', rstride=6, cstride=12)
    
    #right cone
    X,Y,Z, Xc,Yc,Zc = cone(np.radians(alphaR1), np.radians(alphaR2), np.radians(beta))
    ax1.plot_surface(X,Y,Z, alpha=0.3, color='red', rstride=6, cstride=12)
    ax2.plot_surface(X,Y,Z, alpha=0.3, color='red', rstride=6, cstride=12)
    ax3.plot_surface(X,Y,Z, alpha=0.3, color='red', rstride=6, cstride=12)
    ax1.plot_surface(Xc, Yc, Zc, alpha=0.3, color='red', rstride=6, cstride=12)
    ax2.plot_surface(Xc, Yc, Zc, alpha=0.3, color='red', rstride=6, cstride=12)
    ax3.plot_surface(Xc, Yc, Zc, alpha=0.3, color='red', rstride=6, cstride=12)

    R = Rot.from_euler('xyz', [-np.pi/2, 0, np.radians(offset)], degrees=False).as_matrix()
    pts3d_display = np.matmul(R,pts3d.T).T

    plot_pose_3d(ax1, tar=pts3d_display, bones=par_data["vis"]["bones"], limb_id=par_data["vis"]["limb_id"], colors=par_data["vis"]["colors"])
    plot_pose_3d(ax2, tar=pts3d_display, bones=par_data["vis"]["bones"], limb_id=par_data["vis"]["limb_id"], colors=par_data["vis"]["colors"])
    plot_pose_3d(ax3, tar=pts3d_display, bones=par_data["vis"]["bones"], limb_id=par_data["vis"]["limb_id"], colors=par_data["vis"]["colors"])
    
    ax1.set_xlim(-3, 3)
    ax1.set_ylim(-3, 3)
    ax1.set_zlim(-1.5, 1.5)
    ax2.set_xlim(-3, 3)
    ax2.set_ylim(-3, 3)
    ax2.set_zlim(-1.5, 1.5)
    ax3.set_xlim(-2.5, 2.5)
    ax3.set_ylim(-2.5, 2.5)
    ax3.set_zlim(-2, 2)
    #ax1.set_xlabel("X")
    #ax1.set_ylabel("Y")
    #ax1.set_zlabel("Z")
    #ax2.set_xlabel("X")
    #ax2.set_ylabel("Y")
    #ax2.set_zlabel("Z")
    #ax3.set_xlabel("X")
    #ax3.set_ylabel("Y")
    #ax3.set_zlabel("Z")
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    ax1.set_zticklabels([])
    ax2.set_xticklabels([])
    ax2.set_yticklabels([])
    ax2.set_zticklabels([])
    ax3.set_xticklabels([])
    ax3.set_yticklabels([])
    ax3.set_zticklabels([])
    
    plt.savefig('angle_invariant.svg')

    plt.show()

interact(three_d_plotter, 
         offset = widgets.FloatSlider(
    min=-180,
    max=180,
    step=1,
    description='Long. axis (align with +x):',
    value=45
),
         alphaL1 = widgets.FloatSlider(
    min=-180,
    max=0,
    step=10,
    description='Left yaw min:',
    value=-150
),
         alphaL2 = widgets.FloatSlider(
    min=-180,
    max=0,
    step=10,
    description='Left yaw max:',
    value=-60
),
        alphaR1 = widgets.FloatSlider(
    min=0,
    max=180,
    step=10,
    description='Right yaw min:',
    value=60
),
         alphaR2 = widgets.FloatSlider(
    min=0,
    max=180,
    step=10,
    description='Right yaw max:',
    value=150
),
         beta = widgets.FloatSlider(
    min=0,
    max=90,
    step=1,
    description='Roll:',
    value=20
))

interactive(children=(FloatSlider(value=45.0, description='Long. axis (align with +x):', max=180.0, min=-180.0…

<function __main__.three_d_plotter(offset, alphaL1, alphaL2, alphaR1, alphaR2, beta)>